In [1]:
# ! pip install mediapipe

In [2]:
import cv2
import mediapipe as mp
import time
import matplotlib.pyplot as plt


In [7]:
class FaceMeshDetector():
    
    def __init__(self, static_image_mode= False, max_num_faces= 1, min_detection_confidence= 0.5, min_tracking_confidence= 0.5):
        
        self.static_image_mode= static_image_mode
        self.max_num_faces = max_num_faces
        self.min_detection_confidence= min_detection_confidence
        self.min_tracking_confidence= min_tracking_confidence
                     
                       
        # Initialize mediapipe face detection and landmark models
        self.mpDraw= mp.solutions.drawing_utils  # creates line between detected points
        self.mpFaceMesh= mp.solutions.face_mesh
        self.faceMesh= self.mpFaceMesh.FaceMesh(max_num_faces= 3)
        self.drawSpec= self.mpDraw.DrawingSpec(thickness= 1, circle_radius= 2)
    
    def findFaceMesh(self, img, draw= True):

        # convert to rgb
        self.imgRGB= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        self.results= self.faceMesh.process(self.imgRGB)

        faces= []
        if self.results.multi_face_landmarks:
            for faceLms in self.results.multi_face_landmarks:
                self.mpDraw.draw_landmarks(img, faceLms, self.mpFaceMesh.FACEMESH_CONTOURS, self.drawSpec, self.drawSpec)

                face= []
                for id, lm in enumerate(faceLms.landmark):
    #                 print(id, lm)
                    ih, iw, ic= img.shape
                    x, y= int(lm.x*iw), int(lm.y*ih)
#                     print(x,y)
#                     cv2.putText(img, str(id), (x, y), cv2.FONT_HERSHEY_PLAIN,0.5, (0,255,0), 1)

                    face.append([x, y])
                faces.append(face)
                
        return img, faces


def main():
    # initialize the video
#     cap= cv2.VideoCapture(r"vid_1.mp4")
    cap= cv2.VideoCapture(0)

    # checking if video capture was sucessful
    if not cap.isOpened():
        print("Unable to find Video file")
        exit()

    pTime= 0
    detector= FaceMeshDetector()
    # Loop to read frames from the video
    while True:

        # Read a frame from the video
        success, img= cap.read()   # here img is BGR
        
        img, faces= detector.findFaceMesh(img, False)
        
#         if len(faces)!=0:
#             print(len(faces))
#             print(faces[0])

        # write frame rate
        cTime= time.time()
        fps= 1/(cTime-pTime)   # c for current time, p for previous time
        pTime= cTime
        
        # check if frame was read sucessfully
        if  success:  
            cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
            cv2.imshow("Image", img)

            if cv2.waitKey(1) == ord('q'):
                break
    
    #release the capture object
    cap.release()

    #close all active windows
    cv2.destroyAllWindows()
    
    
if __name__== '__main__':
    main()


